Medway - Local Election Results - 2023
======================================

In [ ]:
%matplotlib inline

from IPython.display import display, HTML
from matplotlib import pyplot as plt
import contextily as cx
import geopandas as gpd
import pandas as pd
import re
import requests
import seaborn as sns

### TODO

[ ] Rejection reasons - add to df_ward  

### 0. Config

In [ ]:
# Paths
path_data = 'data'


In [ ]:
# Ward seats
ward_seats = {
    "All Saints": 1,
    "Chatham Central and Brompton": 3,
    "Cuxton, Halling and Riverside": 2,
    "Fort Horsted": 1,
    "Fort Pitt": 3,
    "Gillingham North": 3,
    "Gillingham South": 3,
    "Hempstead and Wigmore": 2,
    "Hoo St Werburgh and High Halstow": 3,
    "Lordswood and Walderslade": 3,
    "Luton": 2,
    "Princes Park": 2,
    "Rainham North": 3,
    "Rainham South East": 3,
    "Rainham South West": 2,
    "Rochester East and Warren Wood": 3,
    "Rochester West and Borstal": 3,
    "St Mary's Island": 1,
    "Strood North and Frindsbury": 3,
    "Strood Rural": 3,
    "Strood West": 3,
    "Twydall": 2,
    "Watling": 3,
    "Wayfield and Weeds Wood": 2
 }
assert sum([x for x in ward_seats.values()]) == 59

In [ ]:
# Party map
map_party = {
       "Conservative Party candidate": "Conservative",
       "Labour and Co-operative Party": "Labour",
       "Labour Party": "Labour",
       "Local Conservatives": "Conservative",
}

In [ ]:
# Colours map
x = sns.color_palette("Paired")
print(x.as_hex())
display(x)

map_colors = {
       "Christian Peoples Alliance": "#ffff99",
       "Conservative Party candidate": "#1f78b4",
       "Conservative": "#1f78b4",
       "Green Party": "#33a021",
       "Heritage Party": "#fdbf6f",
       "Independent": "#ff7f00",
       "Labour and Co-operative Party": "#fb9a99",
       "Labour Party": "#e31a1c",
        "Labour": "#e31a1c",
       "Liberal Democrats": "#cab2d6",
       "Local Conservatives": "#a6cee3",
       "Reform UK": "#b15928",
       "Social Democratic Party": "#6a3d9a"
}



### 1. Load Data

In [ ]:
### 1. Load data
df_wards = pd.read_csv('data/wards.csv')
df_results = pd.read_csv('data/results.csv')

### 2. Analysis

In [ ]:
def plot_ward(df):
    (
        df
        .plot(kind='scatter', 
              y='surname', 
              x='Number of votes', 
              figsize=(8,6), 
              s=200,
              c=df['party'].map(map_colors))
    )
    plt.gca().invert_yaxis()
    plt.show()

    return df

for ward in df_results['ward'].unique():
    display(HTML(f'<h3>{ward}</h3>'))
    
    display(HTML(f'<h4>Overview</h4>'))
    print(f'Seats: {df_wards.loc[df_wards["ward"] == ward]["ward_seats"].item()}')
    
    display(HTML(f'<h4>Candidate results - {ward} - table</h4>'))
    df = df_results[df_results['ward'] == ward].sort_values(by='Number of votes', ascending=False)
    display(df)
    display(HTML(f'<h4>Candidate results - {ward}</h4>'))
    plot_ward(df)
    
    display(HTML(f'<hr/>'))
    
    

In [ ]:
df_results[df_results['Surname'] == 'Field']

In [ ]:
df_results.sort_values(by='Number of votes', ascending=False).head(15)

### Which party got the most votes overall?

In [ ]:


display(df_results.groupby(['party']).sum().sort_values(by='Number of votes', ascending=False))

_df = (
    df_results.groupby(['party']).sum().sort_values(by='Number of votes', ascending=True)
    .reset_index()
)

_df.plot(kind='scatter', x='Number of votes', y='party', 
         s=200,
         c=_df['party'].map(map_colors),
        figsize=(10,6),
        title='Medway Local Elections 2023 - total party votes (absolute votes)')

plt.show()

#### 3.2 Analysis - Wards

In [ ]:
df_results_by_ward = (
    df_results.groupby(['ward', 'party']).sum()
    .reset_index()
    .sort_values(by='ward', ascending=False)
)


df_results_by_ward.plot(kind='scatter', x='Number of votes', y='ward', figsize=(10,8), 
                        s=100,c=df_results_by_ward['party'].map(map_colors))
plt.show()

In [ ]:
df_results_by_ward = (
    df_results.groupby(['ward', 'party'])['Number of votes'].sum()
    .reset_index()
    .assign(**{'number_of_votes_norm': lambda _df: _df['Number of votes'] / _df.groupby(['ward'])['Number of votes'].transform('sum')})
    .sort_values(by='ward', ascending=False)
)

df_results_by_ward.plot(kind='scatter', x='number_of_votes_norm', y='ward', figsize=(10,8), s=100, c=df_results_by_ward['party'].map(map_colors))
plt.show()

In [ ]:
### Top parties in wards
df_results_by_ward.sort_values('number_of_votes_norm', ascending=False).groupby('ward').head(1).sort_values(by=['number_of_votes_norm'], ascending=[True])

### How big is the electorate?

In [ ]:
print(f"Electorate: {df_wards['electorate'].sum():,}")

### Which wards had the best turnout?

In [ ]:
(
    df_wards.sort_values('turnout', ascending=False)
    [['ward', 'ballot_papers_verified', 'electorate', 'turnout']]
    .round({'turnout': 2})
)

### Which wards mostly use postal votes?

In [ ]:
(
    df_wards
    .sort_values(by='postal_ballot_perc', ascending=False)
    [['ward', 'verified_postal_ballot_papers', 'ballot_papers_verified', 'postal_ballot_perc']]
)

### How many ward seats are there to registered electors?

In [ ]:
(
    df_wards
    .assign(**{'ward_seats_per_electorate': lambda _df: _df['electorate'] / _df['ward_seats']})
    .sort_values(by='ward_seats_per_electorate', ascending=False)
    [['ward', 'electorate', 'ward_seats', 'ward_seats_per_electorate']]
    .round({'ward_seats_per_electorate': 1})
)

### Map ward top party by vote

Which party received the most votes in each ward?

In [ ]:
map_ward_names = {
    'Lordswood & Walderslade': 'Lordswood and Walderslade',
    'Hempstead & Wigmore': 'Hempstead and Wigmore',
    'Wayfield & Weeds Wood': 'Wayfield and Weeds Wood',
    'Cuxton, Halling & Riverside': 'Cuxton, Halling and Riverside',
    'Rochester East & Warren Wood': 'Rochester East and Warren Wood',
    'Rochester West & Borstal': 'Rochester West and Borstal',
    'Chatham Central & Brompton': 'Chatham Central and Brompton',
    'Hoo St Werburgh & High Halstow': 'Hoo St Werburgh and High Halstow',
    'Strood North & Frindsbury': 'Strood North and Frindsbury'
}

gpd_medway = (
    gpd.read_file('data/maps/Medway.geojson')
    .drop(columns={'OBJECTID', 'Ward_name'})
    .rename(columns={
        'Name': 'ward_name',
        'No_of_coun': 'councillors',
        'Current_el': 'electorate_current',
        'Forecast_e': 'electorate_forecast'
    })
    .assign(**{
        'ward_name': lambda x: x['ward_name'].replace(to_replace=map_ward_names),
    })
)


df_ward_top_party = (
    df_results_by_ward
    .sort_values('number_of_votes_norm', ascending=False)
    .groupby('ward').head(1).
    sort_values(by=['number_of_votes_norm'], ascending=[True])
    .assign(**{'ward_color': lambda x: x['party'].replace(map_colors)})
)


gdp_medway_top_party = (
    gpd_medway
    .merge(
        df_ward_top_party,
        left_on='ward_name',
        right_on='ward', how='left')
).to_crs(epsg=3857)

ax = gdp_medway_top_party.plot(figsize=(18,20), color=gdp_medway_top_party['ward_color'], edgecolor="black", alpha=0.5)
cx.add_basemap(ax,crs=gdp_medway_top_party.crs, source=cx.providers.Stamen.TonerLite)
# cx.add_basemap(ax,crs=gdp_medway_top_party.crs, source=cx.providers.Stamen.TonerLabels, zoom=12)

# gdp_medway_top_party
plt.axis('off')
plt.show()

display(df_ward_top_party)

### Voter turnout choropleth


In [ ]:
gdp_medway_wards = (
    gpd_medway
    .merge(
        df_wards,
        left_on='ward_name',
        right_on='ward', how='left')
).to_crs(epsg=3857)

ax = gdp_medway_wards.plot(figsize=(18,20), column='turnout', edgecolor="black", alpha=0.5, cmap='YlGn')
cx.add_basemap(ax,crs=gdp_medway_wards.crs, source=cx.providers.Stamen.TonerLite)

plt.axis('off')
plt.show()

display(gdp_medway_wards[['ward', 'turnout']])

In [ ]:
gdp_medway_wards = (
    gpd_medway
    .merge(
        df_wards,
        left_on='ward_name',
        right_on='ward', how='left')
).to_crs(epsg=3857)

ax = gdp_medway_wards.plot(figsize=(18,20), column='postal_ballot_perc', edgecolor="black", alpha=0.5, cmap='Blues')
cx.add_basemap(ax,crs=gdp_medway_wards.crs, source=cx.providers.Stamen.TonerLite)

plt.axis('off')
plt.show()

display(gdp_medway_wards[['ward', 'postal_ballot_perc']].sort_values(by='postal_ballot_perc', ascending=False))